In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/kaggle/input/amazon-ml-challenge-ds/68e8d1d70b66d_student_resource/student_resource/dataset/train.csv')
df_t = pd.read_csv('/kaggle/input/amazon-ml-challenge-ds/68e8d1d70b66d_student_resource/student_resource/dataset/test.csv')

In [4]:
pip install transformers torch pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 2.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 32.3 MB/s eta 0:00:00
  Attempting uninstall: 

In [5]:
# --- Preprocessing ---
# Handle potential missing values in text (if any)
df['catalog_content'].fillna("missing", inplace=True)
df_t['catalog_content'].fillna("missing", inplace=True)

# Log transform the target variable 'price'
# np.log1p(x) is equivalent to np.log(x + 1), which handles prices of 0
df['price_log'] = np.log1p(df['price'])

print("Data loaded and preprocessed.")
print(f"Train data shape: {df.shape}")

Data loaded and preprocessed.
Train data shape: (75000, 5)


/tmp/ipykernel_37/1271518422.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['catalog_content'].fillna("missing", inplace=True)
/tmp/ipykernel_37/1271518422.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [6]:
import torch
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader

# --- Configuration ---
MODEL_NAME = 'bert-base-uncased' # A good starting point
MAX_LEN = 128 # Max sequence length for BERT; adjust based on your text length
BATCH_SIZE = 64

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# --- Create a custom PyTorch Dataset ---
class PricePredictionDataset(Dataset):
    def __init__(self, texts, prices, tokenizer, max_len):
        self.texts = texts
        self.prices = prices
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        
        # Tokenize the text
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        # Prepare item dictionary
        item = {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs['token_type_ids'].flatten()
        }
        
        # If prices are available (for training/validation)
        if self.prices is not None:
            item['targets'] = torch.tensor(self.prices[idx], dtype=torch.float)

        return item

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
from transformers import BertForSequenceClassification
from torch.optim import AdamW 
# Set device to GPU if available, else CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Load the model and configure it for regression (num_labels=1)
model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=1 # This is the key for regression!
)

# Move the model to the selected device
model.to(device)

print(f"Model loaded on: {device}")

2025-10-12 16:43:33.772566: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760287413.944308      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760287413.996104      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded on: cuda


In [8]:
import numpy as np

def smape(y_true, y_pred):
    """
    Calculates the Symmetric Mean Absolute Percentage Error (SMAPE).
    
    Args:
        y_true (np.array): Array of true values.
        y_pred (np.array): Array of predicted values.
        
    Returns:
        float: The SMAPE score.
    """
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    
    # Handle the case where both y_true and y_pred are zero
    # to avoid division by zero.
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    
    return np.mean(ratio) * 100

In [ ]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from torch.utils.data import DataLoader

# Make sure device is set
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Split data into training and validation sets
train_texts, val_texts, train_prices, val_prices = train_test_split(
    df['catalog_content'].values,
    df['price_log'].values,
    test_size=0.2,
    random_state=55
)

# Create Dataset and DataLoader objects (as before)
train_dataset = PricePredictionDataset(train_texts, train_prices, tokenizer, MAX_LEN)
val_dataset = PricePredictionDataset(val_texts, val_prices, tokenizer, MAX_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Training Setup
EPOCHS = 7
LEARNING_RATE = 3e-5
PATIENCE = 3  # Early stopping patience

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
model.to(device)

best_val_loss = float('inf')
patience_counter = 0

for epoch in range(EPOCHS):
    model.train()
    total_train_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['targets'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=targets)
        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_train_loss = total_train_loss / len(train_loader)

    # Validation
    model.eval()
    total_val_loss = 0
    val_preds_log = []
    val_targets_log = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=targets)
            loss = outputs.loss
            total_val_loss += loss.item()

            val_preds_log.extend(outputs.logits.squeeze().cpu().numpy())
            val_targets_log.extend(targets.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)

    # Convert predictions and targets back to original price scale
    val_preds_orig = np.expm1(val_preds_log)
    val_targets_orig = np.expm1(val_targets_log)

    # Calculate validation SMAPE on original scale
    val_smape = smape(val_targets_orig, val_preds_orig)

    print(f"Epoch {epoch + 1}/{EPOCHS} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val SMAPE: {val_smape:.4f}%")

    # Early stopping & save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pth")
        print("Saved best model.")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print(f"Early stopping triggered after {epoch + 1} epochs.")
            break

print("Training complete!")

Using device: cuda
Epoch 1/7 | Train Loss: 0.6255 | Val Loss: 0.4605 | Val SMAPE: 51.4540%
Saved best model.
Epoch 2/7 | Train Loss: 0.4381 | Val Loss: 0.4291 | Val SMAPE: 49.0625%
Saved best model.
Epoch 3/7 | Train Loss: 0.3501 | Val Loss: 0.4274 | Val SMAPE: 47.8123%
Saved best model.
Epoch 4/7 | Train Loss: 0.2777 | Val Loss: 0.4217 | Val SMAPE: 46.9889%
Saved best model.
Epoch 5/7 | Train Loss: 0.2208 | Val Loss: 0.4595 | Val SMAPE: 48.5625%
Epoch 6/7 | Train Loss: 0.1739 | Val Loss: 0.4410 | Val SMAPE: 47.2941%


In [ ]:
# Create test dataset and dataloader
test_texts = df_t['catalog_content'].values
# Pass None for prices as they are not available for the test set
test_dataset = PricePredictionDataset(test_texts, None, tokenizer, MAX_LEN)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --- Prediction Loop ---
model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions.extend(outputs.logits.squeeze().cpu().numpy())

# Convert log predictions back to original price scale
final_predictions = np.expm1(predictions)

# Ensure prices are not negative (a safe-guard)
final_predictions[final_predictions < 0] = 0 

# Create submission file
submission_df = pd.DataFrame({
    'sample_id': df_t['sample_id'],
    'price': final_predictions
})

submission_df.to_csv('test_out.csv', index=False)
print(" Submission file created successfully!")